# Pipeline Implementation

This code will contain the implementation for our pipeline that combines both our object detection and our traffic prediction network

In [1]:
from object_detection.callable_implementation import write_csv
from object_detection.callable_implementation import process_image
from object_detection.callable_implementation import get_dt_info
from object_detection.callable_implementation import load_model
from object_detection.callable_implementation import load_tensor_img
from object_detection.render_video import images_to_video_ffmpeg

import requests
import cv2
import numpy as np
import imutils
import time
import os
import threading
from queue import Queue
import shutil

In [2]:
# load the model
model, vehicle_classes, device = load_model()

In [3]:
# read ip from text file
with open('my_ip.txt', 'r') as f:
    ip = f.read().strip()
print(ip)

192.168.4.111:8080


In [ ]:
import threading
import time
import os
import cv2
import requests
import numpy as np
import imutils
from queue import Queue


RUNNING = True

# Replace the below URL with your own. Make sure to add "/shot.jpg" at the last.
url = f"http://{ip}/shot.jpg"

# Initialize variables
fps = 10  # Desired FPS for the output video
out_dir = "test_images"

# Create the output directory if it doesn't exist
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# Queue for storing images that need to be annotated and displayed
image_queue = Queue()
annotated_image_queue = Queue()

# Function to process the image
def annotate(image_path):
    print(f"Annotating image: {image_path}")
    output_path = 'live_stream'
    image_base_name = os.path.basename(image_path).split('.')[0]
    outfile_name = f'{image_base_name}_annotated.jpg'
    img, img_tensor = load_tensor_img(image_path, device)
    data, out_image = process_image(model, vehicle_classes, img, img_tensor, output_path, outfile_name)
    time, day = get_dt_info()
    data["Time"] = time
    data["Day of the week"] = day
    write_csv(data, "test.csv")
    
    # Save the annotated image
    annotated_image_path = os.path.join(out_dir, outfile_name)
    cv2.imwrite(annotated_image_path, out_image)

    # Put the annotated image path in the annotated queue
    annotated_image_queue.put(annotated_image_path)

# Function to process images from the queue for annotation
def process_annotations():
    while True:
        # Get an image path from the queue
        img_path = image_queue.get()
        if img_path is None:  # Sentinel value to stop the thread
            break
        # Call the annotation function for each image
        annotate(img_path)
        image_queue.task_done()

# Function to display annotated images in a separate thread
def display_annotated_images():
    global RUNNING
    while RUNNING:
        # Get the annotated image path from the queue
        annotated_img_path = annotated_image_queue.get()
        if annotated_img_path is None:  # Sentinel value to stop the thread
            break
        
        # Read and display the annotated image
        annotated_img = cv2.imread(annotated_img_path)
        cv2.imshow("Annotated Image", annotated_img)
        
        # Press Esc key to exit the display window
        if cv2.waitKey(1) == 27:
            RUNNING = False
            break
        annotated_image_queue.task_done()

# Start the annotation processing thread
annotation_thread = threading.Thread(target=process_annotations, daemon=True)
annotation_thread.start()

# Start the display thread
display_thread = threading.Thread(target=display_annotated_images, daemon=True)
display_thread.start()

# Timing control to achieve 30 FPS
prev_time = time.time()

# While loop to continuously fetch data from the URL
count = 0
while RUNNING:
    # Calculate the time difference
    current_time = time.time()

    img_resp = requests.get(url)
    img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8)
    img = cv2.imdecode(img_arr, -1)
    img = imutils.resize(img, width=1000, height=1800)

    # Save the image with a timestamp to ensure uniqueness
    filename = f"{out_dir}/image_{count}.jpg"
    cv2.imwrite(filename, img)

    # Add the image path to the queue for annotation
    image_queue.put(filename)

    # # Display the image on the Android camera window
    # cv2.imshow("Android_cam", img)

    # # Press Esc key to exit
    # if cv2.waitKey(1) == 27:
    #     break

    # Sleep to avoid high CPU usage (sleeping 1ms in case timing isn't perfect)
    time.sleep(1 / fps)
    count += 1
    
cv2.destroyWindow("Annotated Image")
cv2.destroyAllWindows()
# Stop the annotation and display threads gracefully
image_queue.put(None)  # Sentinel to stop the annotation thread
annotation_thread.join()

annotated_image_queue.put(None)  # Sentinel to stop the display thread
display_thread.join()

total_run_time = time.time() - prev_time
num_seconds = int(total_run_time)
# get num images in live_stream folder
num_images = len([name for name in os.listdir("live_stream") if os.path.isfile(os.path.join("live_stream", name))])
estimated_fps = num_images / num_seconds
print(f"Estimated FPS: {estimated_fps}")


Annotating image: test_images/image_0.jpg
Annotating image: test_images/image_1.jpg
Annotating image: test_images/image_2.jpg
Annotating image: test_images/image_3.jpg
Annotating image: test_images/image_4.jpg
Annotating image: test_images/image_5.jpg
Annotating image: test_images/image_6.jpg
Annotating image: test_images/image_7.jpg
Annotating image: test_images/image_8.jpg
Annotating image: test_images/image_9.jpg
Annotating image: test_images/image_10.jpg
Annotating image: test_images/image_11.jpg
Annotating image: test_images/image_12.jpg
Annotating image: test_images/image_13.jpg
Annotating image: test_images/image_14.jpg
Annotating image: test_images/image_15.jpg
Annotating image: test_images/image_16.jpg
Annotating image: test_images/image_17.jpg
Annotating image: test_images/image_18.jpg
Annotating image: test_images/image_19.jpg
Annotating image: test_images/image_20.jpg
Annotating image: test_images/image_21.jpg
Annotating image: test_images/image_22.jpg
Annotating image: tes

In [ ]:

date_time_info = get_dt_info()
images_to_video_ffmpeg("live_stream", f"live_stream_{date_time_info}.mp4", estimated_fps)

# remove live_stream folder and its contents
shutil.rmtree("live_stream")

2024/11/27 13:28:40.379850 cmd_run.go:1138: WARNING: cannot start document portal: dial unix /run/user/1000/bus: connect: no such file or directory
ffmpeg version n4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix= --prefix=/usr --disable-debug --disable-doc --disable-static --enable-cuda --enable-cuda-sdk --enable-cuvid --enable-libdrm --enable-ffplay --enable-gnutls --enable-gpl --enable-libass --enable-libfdk-aac --enable-libfontconfig --enable-libfreetype --enable-libmp3lame --enable-libnpp --enable-libopencore_amrnb --enable-libopencore_amrwb --enable-libopus --enable-libpulse --enable-sdl2 --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libv4l2 --enable-libvorbis --enable-libvpx --enable-libx264 --enable-libx265 --enable-libxcb --enable-libxvid --enable-nonfree --enable-nvenc --enable-omx --enable-openal --enable-opencl --enable-runtime-cpudetect --enable-shared --enable-vaapi --enable-

Video saved as live_stream_('132840', 'Wednesday').mp4


frame=   63 fps= 53 q=-1.0 Lsize=    1366kB time=00:00:06.00 bitrate=1864.4kbits/s speed=5.08x    
video:1364kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.097784%
[libx264 @ 0x555ee7e51840] frame I:3     Avg QP:21.99  size: 51144
[libx264 @ 0x555ee7e51840] frame P:48    Avg QP:20.34  size: 20592
[libx264 @ 0x555ee7e51840] frame B:12    Avg QP:24.98  size: 21203
[libx264 @ 0x555ee7e51840] consecutive B-frames: 65.1% 28.6%  0.0%  6.3%
[libx264 @ 0x555ee7e51840] mb I  I16..4:  3.3% 72.6% 24.1%
[libx264 @ 0x555ee7e51840] mb P  I16..4:  4.1% 41.5%  6.7%  P16..4: 24.3%  8.0%  4.5%  0.0%  0.0%    skip:10.8%
[libx264 @ 0x555ee7e51840] mb B  I16..4:  1.3% 11.8%  5.3%  B16..8: 37.0% 12.3%  2.9%  direct:13.7%  skip:15.6%  L0:43.8% L1:44.5% BI:11.7%
[libx264 @ 0x555ee7e51840] 8x8 transform intra:77.6% inter:76.8%
[libx264 @ 0x555ee7e51840] coded y,uvDC,uvAC intra: 64.8% 57.0% 38.6% inter: 38.2% 34.8% 2.8%
[libx264 @ 0x555ee7e51840] i16 v,h,dc,p: 26% 17% 12% 45%
